In [23]:
# %matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [24]:
# import libraries
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [25]:
# import sqlalchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [26]:
# connect to datasource
engine = create_engine('sqlite:///Resource/hawaii.sqlite')

In [27]:
Base = automap_base()

In [28]:
# reflect database into model
Base.prepare(engine, reflect=True)

Station = Base.classes.station
Measurement = Base.classes.measurement

In [31]:
session = Session(engine)

In [69]:
# get the max date for the precipitation measurement
max_date = session.query(func.max(Measurement.date)).scalar()

# calculate the date 12 months before the last date
query_start_date = dt.datetime.fromisoformat(max_date) - dt.timedelta(days=366)

# write query results to dataframe
precip_df = pd.DataFrame(session.query(Measurement.prcp, Measurement.date).filter(Measurement.date >= query_start_date).all())
precip_df.set_index('date', inplace=True)
precip_df.sort_values('date')
precip_df.head()

,prcp
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00
